# Assignment #1: Building an inverted index
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program that collects all the words from a set of documents
* Build an index from the words
* Represent a document using the Tf.Idf values
* Write a short report of 1 to 2 pages on the assignment
* Read a short text on an industrial system

## Description of the assignment

### Outline

In this lab, you will build an indexer to index all the words in a corpus. Conceptually, an index consists of rows with one word per row and the list of files and positions, where this word occurs. Such a row is called a _posting list_. You will encode the position of a word by the number of characters from the start of the file.
<pre>
word1: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
word2: file_name pos1 pos2 pos3... file_name pos1 pos2 ...
...
</pre>

### Indexing one file

#### Description

<p>Write a program that reads one document <tt>file_name.txt</tt> and outputs an index file:
            <tt>file_name.idx</tt>:
        </p>
        <ol>
            <li>The index file will contain all the unique words in the document,
                where each word is associated with the list of its positions in the document.
            </li>
            <li>You will represent this index as a dictionary, where the keys will be the words, and
                the values, the lists of positions
            </li>
            <li>As words, you will consider all the strings of letters that you will set in lower case.
                You will not index the rest (i.e. numbers, punctuations, or symbols).
            </li>
            <li>To extract the words, use Unicode regular expressions. Do not use <tt>\w+</tt>,
                for instance, but the Unicode equivalent.
            </li>
            <li>The word positions will correspond to the number of characters from the beginning of the file.
                (The word offset from the beginning)
            </li>
            <li>You will use the <tt>finditer()</tt> method to find the positions of the words.
                This will return you match objects,
                where you will get the matches and the positions with
                the <tt>group()</tt> and <tt>start()</tt> methods.
            </li>
            <li>You will use the pickle package to write your dictionary in an file,
                see <a href="https://wiki.python.org/moin/UsingPickle">https://wiki.python.org/moin/UsingPickle</a>.
            </li>
        </ol>

Below is an excerpt of the index of the `bannlyst.txt` text for the words <i>gjord</i>, <i>uppklarnande</i>, and <i>stjärnor</i>. The data is stored in a dictionary:

<pre>
{...
'gjord': [8600, 183039, 220445],
'uppklarnande': [8617],
'stjärnor': [8641], ...
}
</pre>
where the word <i>gjord</i> occurs three times in the text at positions 8600, 183039, and 220445, <i>uppklarnande</i>, once at position 8617, and <i>stjärnor</i>, once at position 8641.

#### Writing a tokenizer 

Write a Unicode regular expression to find words defined as sequences of letters.

In [3]:
# Extract all words from sentence using regex expression
def get_words(filename, reg=re.compile('\p{L}+')):
    return re.findall(reg, read_file(filename).lower())

In [4]:
re.findall(regex, 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa')

['En',
 'gång',
 'hade',
 'de',
 'på',
 'Mårbacka',
 'en',
 'barnpiga',
 'som',
 'hette',
 'Back',
 'Kajsa']

Using `regex`, write `tokenize(text)` function to tokenize a text. Return their positions.

In [5]:
# Returns scanner objects that can be used to extract position information
def tokenize(text, regex=re.compile('\p{L}+')):
    return re.finditer(regex, text)

In [ ]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.')
list(tokens)

#### Extracting indices

Write a `text_to_idx(words)` function to extract the indices from the list of tokens (words). Return a dictionary, where the keys will be the tokens (words), and the values a list of positions.

In [7]:
# Constructs a dictionary with words as keys and character offset from beginning of document as values.
def text_to_idx(tokens):
    ddict = defaultdict(list)
    for token in tokens:
        ddict[token.group()].append(token.start())
    return ddict

In [8]:
tokens = tokenize('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.'.lower().strip())
text_to_idx(tokens)

{'en': [0, 28],
 'gång': [3],
 'hade': [8],
 'de': [13],
 'på': [16],
 'mårbacka': [19],
 'barnpiga': [31],
 'som': [41],
 'hette': [45],
 'back': [51],
 'kajsa': [56]}

#### Reading one file

Read one file, _Mårbacka_, `marbacka.txt`, set it in lowercase, tokenize it, and index it. Call this index `idx`

In [9]:
text = read_file('marbacka.txt').lower()
tokens = tokenize(text)
idx = text_to_idx(tokens)

#### Saving the index

Save your index in a file so that you can reuse it. Use the pickle module.

In [11]:
pickle.dump(idx, open('marbacka_idx', 'wb'))
idx = pickle.load(open('marbacka_idx', 'rb'))

### Reading the content of a folder

Write a `get_files(dir, suffix)` function that reads all the files in a folder with a specific `suffix` (txt). You will need the Python `os` package, see <a href="https://docs.python.org/3/library/os.html">https://docs.python.org/3/library/os.html</a>. You will return the file names in a list.

You can reuse this function:

In [14]:
def get_files(dir, suffix):
    """
    Returns all the files in a folder ending with suffix
    :param dir:
    :param suffix:
    :return: the list of file names
    """
    files = []
    for file in os.listdir(dir):
        if file.endswith(suffix):
            files.append(file)
    return files

### Creating a master index

Complete your program with the creation of master index, where you will associate each word of the corpus with the files, where it occur and its positions: a posting list
Below is an except of the master index with the words <i>samlar</i> and <i>ände</i>:

In [ ]:
{'samlar':
            {'troll.txt': [641880, 654233],
            'nils.txt': [51805, 118943],
            'osynliga.txt': [399121],
            'gosta.txt': [313784, 409998, 538165]},
 'ände':
            {'troll.txt': [39562, 650112],
            'kejsaren.txt': [50171],
            'marbacka.txt': [370324],
            'nils.txt': [1794],
            'osynliga.txt': [272144]}
}

The word <i>samlar</i>, for instance, occurs three times in the gosta text at positions
            313784, 409998, and 538165.

In [17]:
all_files = get_files(folder_path, '.txt')

master_index = defaultdict(dict)
for file in all_files:
    text = read_file(file).lower()
    tokens = tokenize(text)
    idx = text_to_idx(tokens)
    
    # Insert into master dictionary where the word is the key and the value is a dictionary with the
    # file as the key and value is the indexes of occurrences
    for key, value in idx.items():
        master_index[key][file] = value

Save your master index in a file and read it again

In [20]:
pickle.dump(master_index, open('master_index', 'wb'))
master_index = pickle.load(open('master_index', 'rb'))

#### Concordances

Write a `concordance(word, master_index, window)` function to extract the concordances of a `word` within a window of `window` characters

In [22]:
## Get the concordances (contexts) of a certain word
def concordance(word, master_index, corpus_files, window_size):
    occurences = []
    for file in master_index[word]:
        if file in corpus_files:
            text = read_file(file).lower()
            for occurence in master_index[word][file]:
                center = occurence + len(word) // 2
                shift = window_size // 2
                occurences.append(text[center - shift: center + shift])
            
    return occurences

### Representing Documents with tf-idf

Once you have created the index, you will represent each document in your corpus as a dictionary. The keys of these dictionaries will be the words and you will define the value of a word with the tf-idf metric: 
1. Read the description of the tf-idf measure on Wikipedia (<a href="https://en.wikipedia.org/wiki/Tf%E2%80%93idf">https://en.wikipedia.org/wiki/Tf-idf</a>)
2. After reading the description, you probably realized that there are multiple definitions of tf-idf. In this assignment, 
 * Tf will be the relative frequency of the term in the document and 
 * idf, the logarithm base 10 of the inverse document frequency.
        



Conceptually, the tf-idf representation is a vector. In your program, you will keep this idea and use all the words in the corpus as keys: Each dictionary will include all the words of the corpus as keys. The value of the key is then possibly 0, meaning that the word in not in the document or is in all the documents as for the word `nils` in `gosta.tx`. 

As further work, you may think of optimizing this part.

In [24]:
"""
"tf-idf" basically assigns an importance weight to each word in a file based on how 
common the word is in general, and how often it shows up in other files. For a word 
to be assigned a large weight, the word should be used frequently in the file in question, 
but barely mentioned at all in other files.
"""
def tf_idf(file, corpus_files, word, total_words, master_index):
    tf = len(master_index[word].get(file, [])) / total_words[file]
    
    file_occurences = sum(1 for file in corpus_files if file in master_index[word])
    idf = math.log10(len(corpus_files) / file_occurences)
    
    return tf * idf

### Comparing Documents

Using the cosine similarity, compare all the pairs of documents with their tf-idf representation and present your results in a table. You will include this table in your report.

#### Cosine similarity

Write a function computing the cosine similarity between two documents

In [27]:
def cosine_similarity(file1, file2, all_words, tfidf):
    dot_product = sum(tfidf[file1].get(word, 0) * tfidf[file2].get(word, 0) for word in all_words)
    
    norm_f1 = math.sqrt(sum(tfidf[file1].get(word, 0) ** 2 for word in all_words))
    norm_f2 = math.sqrt(sum(tfidf[file2].get(word, 0) ** 2 for word in all_words))
    return dot_product / (norm_f1 * norm_f2)


#### Similarity matrix

Compute the similarity matrix between the documents of the corpus. While computing the similarities, you will record the two most similar documents that you will call `most_sim_doc1` and `most_sim_doc2`.

In [ ]:
def similarity_matrix(files, all_words, tfidf):
    sim_mat = [[0 for file in files] for file in files]
    for i, file1 in enumerate(files):
        for j, file2 in enumerate(files):
            sim_mat[i][j] = cosine_similarity(file1, file2, all_words, tfidf)
            
    return sim_mat

Give the name of the two novels that are the most similar.

In [29]:
print("Most similar:", most_sim_doc1, most_sim_doc2, "Similarity:", max_similarity)

Most similar: herrgard.txt jerusalem.txt Similarity: 0.3706894238733847
